### 决策树

#### 1.信息熵

In [2]:
def get_dataset():
    """
    获得数据集
    """
    data = [
        [0, 0, 0, 0, 'no'],  #数据集
        [0, 0, 0, 1, 'no'],
        [0, 1, 0, 1, 'yes'],
        [0, 1, 1, 0, 'yes'],
        [0, 0, 0, 0, 'no'],
        [1, 0, 0, 0, 'no'],
        [1, 0, 0, 1, 'no'],
        [1, 1, 1, 1, 'yes'],
        [1, 0, 1, 2, 'yes'],
        [1, 0, 1, 2, 'yes'],
        [2, 0, 1, 2, 'yes'],
        [2, 0, 1, 1, 'yes'],
        [2, 1, 0, 1, 'yes'],
        [2, 1, 0, 2, 'yes'],
        [2, 0, 0, 0, 'no']
    ]
    labels = ['年龄', '有工作', '有自己的房子', '信贷情况']  #分类属性
    return data, labels

In [3]:
dataset, labels = get_dataset()
labels

['年龄', '有工作', '有自己的房子', '信贷情况']

In [3]:
import math


def get_entropy(dataset):
    """
    根据给定的数据集计算经验熵
    """
    nums_dataset = len(dataset)
    labels = {}
    for data in dataset:
        label = data[-1]
        if label not in labels.keys():
            labels[label] = 0
        labels[label] += 1
    # print(labels)
    # 计算经验熵
    entropy = 0.0
    for key in labels:
        # 概率
        prob = labels[key] / nums_dataset
        entropy -= prob * math.log(prob, 2)
    return entropy

In [53]:
get_entropy(dataset)

{'no': 6, 'yes': 9}


0.9709505944546686

In [4]:
def split_dataset(dataset, axis, value):
    ret_dataset = []
    for feature in dataset:
        if feature[axis] == value:
            reduced_feature = feature[:axis]
            reduced_feature.extend(feature[axis + 1:])
            ret_dataset.append(reduced_feature)
    return ret_dataset

In [8]:
import numpy as np
import collections

dataset = np.array(dataset)
collections.Counter(dataset[:, -1])

Counter({'no': 6, 'yes': 9})

In [13]:
len(dataset[0])

5

In [5]:
def get_info_gain(dataset):
    """
    计算信息增益
    """
    num_dataset=len(dataset)
    num_features = len(dataset[0]) - 1
    entropy = get_entropy(dataset)
    info_gain = {}
    for fe in range(num_features):
        features = [feature[fe] for feature in dataset]  # 取出特征列
        unique_feature = set(features)
        condaition_entropy = 0.0
        for value in unique_feature:
            sub_dataset = split_dataset(dataset, fe, value)
            prob = len(sub_dataset) / float(num_dataset)
            condaition_entropy += prob * get_entropy(sub_dataset)
        info_gain[fe] = entropy - condaition_entropy
        # print(info_gain)
    return info_gain

In [6]:
def get_best_feature(dataset):
    """
    获得最好的特征，即信息增益最大值，以ID3为例
    """
    info_gain=get_info_gain(dataset)
    return sorted(info_gain.items(),key=lambda v:v[1],reverse=True)[0][0]

In [7]:
# %%time
credit_dataset, labels = get_dataset()
get_info_gain(credit_dataset)

{0: 0.08300749985576883,
 1: 0.32365019815155627,
 2: 0.4199730940219749,
 3: 0.36298956253708536}

In [ ]:
info_gain=get_info_gain(credit_dataset)
sorted(info_gain.items(),key=lambda v:v[1],reverse=True)[0][0]

In [45]:
def get_major_class(class_list):
    """
        统计类别的数量，并返回数量最多的类别
    """
    class_count = {}
    for vote in class_list:
        if vote not in class_count.keys():
            class_count[vote] = 0
        class_count[vote] += 1
    return sorted(class_count.items(), key=lambda x: x[1], reverse=True)[0][0]

In [73]:
def get_decision_tree(dataset, labels):
    """
        决策树生成
    """
    class_list = [example[-1] for example in dataset]
    if class_list.count(class_list[0]) == len(class_list):
        return class_list[0]  # 终止条件1：样本都是同一类型
    if len(dataset[0]) == 1:
        return get_major_class(class_list)  # 属性用完或分不开情形，使用后验分布

    best_feature = get_best_feature(dataset)
    print(labels, best_feature)
    best_fea_label = labels[best_feature]
    d_tree = {best_fea_label: {}}
    # 删除已经用于分裂的特征
    del (labels[best_feature])
    # 取出特征值
    feature_values = [fe[best_feature] for fe in dataset]
    unique_feature = set(feature_values)
    for v in unique_feature:
        sub_labels = labels[:]
        sub_dataset = split_dataset(dataset, best_feature, v)
        d_tree[best_fea_label][v] = get_decision_tree(sub_dataset, sub_labels)
    return d_tree

In [76]:
credit_dataset, credit_labels = get_dataset()
get_decision_tree(credit_dataset,credit_labels)

['年龄', '有工作', '有自己的房子', '信贷情况'] 2
['年龄', '有工作', '信贷情况'] 1


{'有自己的房子': {0: {'有工作': {0: 'no', 1: 'yes'}}, 1: 'yes'}}

In [9]:
### 使用库函数优化决策树的算法
import numpy as np
import math
import collections

In [61]:
np_dataset = np.array(credit_dataset)
X, y = np_dataset[:, :-1].astype('int64'), np_dataset[:, -1]
X,y

(array([[0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 1],
        [0, 1, 1, 0],
        [0, 0, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 1],
        [1, 1, 1, 1],
        [1, 0, 1, 2],
        [1, 0, 1, 2],
        [2, 0, 1, 2],
        [2, 0, 1, 1],
        [2, 1, 0, 1],
        [2, 1, 0, 2],
        [2, 0, 0, 0]]),
 array(['no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'yes',
        'yes', 'yes', 'yes', 'yes', 'no'], dtype='<U21'))

In [21]:
entropy = 0.0
feature_dict = collections.Counter(y).values()
nums = sum(feature_dict)
for v in feature_dict:
    prob = v / nums
    entropy -= prob * math.log(prob, 2)
entropy

0.9709505944546686

In [18]:
sum(collections.Counter(y).values())

15

In [ ]:
collections.Counter(y)

In [60]:
np_dataset[:, :-1].astype('int64')


array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 1],
       [0, 1, 1, 0],
       [0, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 1],
       [1, 1, 1, 1],
       [1, 0, 1, 2],
       [1, 0, 1, 2],
       [2, 0, 1, 2],
       [2, 0, 1, 1],
       [2, 1, 0, 1],
       [2, 1, 0, 2],
       [2, 0, 0, 0]])

In [62]:
x0=X[:,0]
x0

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])

In [65]:
np.bincount(X[:,1])

array([10,  5])

In [69]:
y_map={
    "no":0,
    "yes":1
}
np.apply_along_axis(y_map,0,y)

TypeError: 'dict' object is not callable

### numpy 实现


In [1]:
import numpy as np


class Node:
    def __init__(self, left, right, rule):
        self.left = left
        self.right = right
        self.feature = rule[0]
        self.threshold = rule[1]


class Leaf:
    def __init__(self, value):
        """
        `value` is an array of class probabilities if classifier is True, else
        the mean of the region
        """
        self.value = value


class DecisionTree:
    def __init__(
            self,
            classifier=True,
            max_depth=None,
            n_feats=None,
            criterion="entropy",
            seed=None,
    ):
        """
        A decision tree model for regression or classification problems.

        Parameters
        ----------
        classifier : bool (default: True)
            Whether to treat target values as categorical (True) or
            continuous (False)
        max_depth: int (default: None)
            The depth at which to stop growing the tree. If None, grow the tree
            until all leaves are pure.
        n_feats : int (default: None)
            Specifies the number of features to sample on each split. If None,
            use all features on each split.
        criterion : str (default: 'entropy')
            The error criterion to use when calculating splits. When
            `classifier` is False, valid entries are {'mse'}. When `classifier`
            is True, valid entries are {'entropy', 'gini'}.
        seed : int (default: None)
            Seed for the random number generator
        """
        if seed:
            np.random.seed(seed)

        self.depth = 0
        self.root = None

        self.n_feats = n_feats
        self.criterion = criterion
        self.classifier = classifier
        self.max_depth = max_depth if max_depth else np.inf

        if not classifier and criterion in ["gini", "entropy"]:
            raise ValueError(
                "{} is a valid criterion only when classifier = True.".format(
                    criterion))
        if classifier and criterion == "mse":
            raise ValueError(
                "`mse` is a valid criterion only when classifier = False.")

    def fit(self, X, Y):
        """
        Trains a binary decision tree classifier.

        Parameters
        ----------
        X : numpy array of shape (N, M)
            The training data of N examples, each with M features
        Y : numpy array of shape (N,)
            An array of integer labels ranging between [0, n_classes-1] for
            each example in X if `self.classifier`=True else the set of target
            values for each example in X.
        """
        self.n_classes = max(Y) + 1 if self.classifier else None
        self.n_feats = X.shape[1] if not self.n_feats else min(
            self.n_feats, X.shape[1])
        self.root = self._grow(X, Y)

    def predict(self, X):
        """
        Use the trained decision tree to classify or predict the examples in X.

        Parameters
        ----------
        X : numpy array of shape (N, M)
            The training data of N examples, each with M features

        Returns
        -------
        preds : numpy array of shape (N,)
            The integer class labels predicted for each example in X if
            classifier = True, otherwise the predicted target values.
        """
        return np.array([self._traverse(x, self.root) for x in X])

    def predict_class_probs(self, X):
        """
        Use the trained decision tree to return the class probabilities for the
        examples in X.

        Parameters
        ----------
        X : numpy array of shape (N, M)
            The training data of N examples, each with M features

        Returns
        -------
        preds : numpy array of shape (N, n_classes)
            The class probabilities predicted for each example in X
        """
        assert self.classifier, "`predict_class_probs` undefined for classifier = False"
        return np.array([self._traverse(x, self.root, prob=True) for x in X])

    def _grow(self, X, Y):
        # if all labels are the same, return a leaf
        if len(set(Y)) == 1:
            if self.classifier:
                prob = np.zeros(self.n_classes)
                prob[Y[0]] = 1.0
            return Leaf(prob) if self.classifier else Leaf(Y[0])

        # if we have reached max_depth, return a leaf
        if self.depth >= self.max_depth:
            v = np.mean(Y, axis=0)
            if self.classifier:
                v = np.bincount(Y, minlength=self.n_classes) / len(Y)
            return Leaf(v)

        N, M = X.shape
        self.depth += 1
        feat_idxs = np.random.choice(M, self.n_feats, replace=False)

        # greedily select the best split according to `criterion`
        feat, thresh = self._segment(X, Y, feat_idxs)
        l = np.argwhere(X[:, feat] <= thresh).flatten()
        r = np.argwhere(X[:, feat] > thresh).flatten()

        # grow the children that result from the split
        left = self._grow(X[l, :], Y[l])
        right = self._grow(X[r, :], Y[r])
        return Node(left, right, (feat, thresh))

    def _segment(self, X, Y, feat_idxs):
        """
        Find the optimal split rule (feature index and split threshold) for the
        data according to `self.criterion`.
        """
        best_gain = -np.inf
        split_idx, split_thresh = None, None
        for i in feat_idxs:
            vals = X[:, i]
            levels = np.unique(vals)
            thresholds = (levels[:-1] + levels[1:]) / 2
            gains = np.array(
                [self._impurity_gain(Y, t, vals) for t in thresholds])

            if gains.max() > best_gain:
                split_idx = i
                best_gain = gains.max()
                split_thresh = thresholds[gains.argmax()]

        return split_idx, split_thresh

    def _impurity_gain(self, Y, split_thresh, feat_values):
        """
        Compute the impurity gain associated with a given split.

        IG(split) = loss(parent) - weighted_avg[loss(left_child), loss(right_child)]
        """
        global loss
        if self.criterion == "entropy":
            loss = entropy
        elif self.criterion == "gini":
            loss = gini
        elif self.criterion == "mse":
            loss = mse

        parent_loss = loss(Y)

        # generate split
        left = np.argwhere(feat_values <= split_thresh).flatten()
        right = np.argwhere(feat_values > split_thresh).flatten()

        if len(left) == 0 or len(right) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(Y)
        n_l, n_r = len(left), len(right)
        e_l, e_r = loss(Y[left]), loss(Y[right])
        child_loss = (n_l / n) * e_l + (n_r / n) * e_r

        # impurity gain is difference in loss before vs. after split
        ig = parent_loss - child_loss
        return ig

    def _traverse(self, X, node, prob=False):
        if isinstance(node, Leaf):
            if self.classifier:
                return node.value if prob else node.value.argmax()
            return node.value
        if X[node.feature] <= node.threshold:
            return self._traverse(X, node.left, prob)
        return self._traverse(X, node.right, prob)


def mse(y):
    """
    Mean squared error for decision tree (ie., mean) predictions
    """
    return np.mean((y - np.mean(y))**2)


def entropy(y):
    """
    Entropy of a label sequence
    """
    hist = np.bincount(y)
    ps = hist / np.sum(hist)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])


def gini(y):
    """
    Gini impurity (local entropy) of a label sequence
    """
    hist = np.bincount(y)
    N = np.sum(hist)
    return 1 - sum([(i / N)**2 for i in hist])

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.datasets.samples_generator import make_blobs


def test_DecisionTree():
    i = 1
    np.random.seed(12345)
    while True:
        n_ex = np.random.randint(2, 100)
        n_feats = np.random.randint(2, 100)
        max_depth = np.random.randint(1, 5)

        classifier = np.random.choice([True, False])
        if classifier:
            # create classification problem
            n_classes = np.random.randint(2, 10)
            X, Y = make_blobs(n_samples=n_ex,
                              centers=n_classes,
                              n_features=n_feats,
                              random_state=i)
            X, X_test, Y, Y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.3,
                                                    random_state=i)

            # initialize model
            def loss(yp, y):
                return 1 - accuracy_score(yp, y)

            criterion = np.random.choice(["entropy", "gini"])
            mine = DecisionTree(classifier=classifier,
                                max_depth=max_depth,
                                criterion=criterion)
            gold = DecisionTreeClassifier(
                criterion=criterion,
                max_depth=max_depth,
                splitter="best",
                random_state=i,
            )
        else:
            # create regeression problem
            X, Y = make_regression(n_samples=n_ex,
                                   n_features=n_feats,
                                   random_state=i)
            X, X_test, Y, Y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.3,
                                                    random_state=i)

            # initialize model
            criterion = "mse"
            loss = mean_squared_error
            mine = DecisionTree(criterion=criterion,
                                max_depth=max_depth,
                                classifier=classifier)
            gold = DecisionTreeRegressor(criterion=criterion,
                                         max_depth=max_depth,
                                         splitter="best")

        print("Trial {}".format(i))
        print("\tClassifier={}, criterion={}".format(classifier, criterion))
        print("\tmax_depth={}, n_feats={}, n_ex={}".format(
            max_depth, n_feats, n_ex))
        if classifier:
            print("\tn_classes: {}".format(n_classes))

        # fit 'em
        mine.fit(X, Y)
        gold.fit(X, Y)

        # get preds on training set
        y_pred_mine = mine.predict(X)
        y_pred_gold = gold.predict(X)

        loss_mine = loss(y_pred_mine, Y)
        loss_gold = loss(y_pred_gold, Y)

        # get preds on test set
        y_pred_mine_test = mine.predict(X_test)
        y_pred_gold_test = gold.predict(X_test)

        loss_mine_test = loss(y_pred_mine_test, Y_test)
        loss_gold_test = loss(y_pred_gold_test, Y_test)

        try:
            np.testing.assert_almost_equal(loss_mine, loss_gold)
            print("\tLoss on training: {}".format(loss_mine))
        except AssertionError as e:
            print("\tTraining losses not equal:\n{}".format(e))

        try:
            np.testing.assert_almost_equal(loss_mine_test, loss_gold_test)
            print("\tLoss on test: {}".format(loss_mine_test))
        except AssertionError as e:
            print("\tTest losses not equal:\n{}".format(e))
        i += 1

In [76]:
test_DecisionTree()

Trial 1
	Classifier=False, criterion=mse
	max_depth=1, n_feats=3, n_ex=31
	Loss on training: 8709.329410399534
	Loss on test: 4103.533167451686
Trial 2
	Classifier=True, criterion=gini
	max_depth=2, n_feats=31, n_ex=36
	n_classes: 3
	Loss on training: 0.0
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.0
 DESIRED: 0.09090909090909094
Trial 3
	Classifier=False, criterion=mse
	max_depth=2, n_feats=25, n_ex=60
	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 15854.344961985342
 DESIRED: 11995.845959423437
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 22374.86521889598
 DESIRED: 13456.376936406066
Trial 4
	Classifier=False, criterion=mse
	max_depth=2, n_feats=88, n_ex=98
	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 18341.679068928388
 DESIRED: 13730.891900511522
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 30131.600387285824
 DESIRED: 

	Loss on training: 0.0
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.033333333333333326
 DESIRED: 0.0
Trial 38
	Classifier=False, criterion=mse
	max_depth=3, n_feats=38, n_ex=51
	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 16876.866489142303
 DESIRED: 4231.498791725337
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 30498.659249724013
 DESIRED: 58750.975472303806
Trial 39
	Classifier=True, criterion=entropy
	max_depth=2, n_feats=31, n_ex=63
	n_classes: 5
	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.38636363636363635
 DESIRED: 0.15909090909090906
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.42105263157894735
 DESIRED: 0.26315789473684215
Trial 40
	Classifier=True, criterion=gini
	max_depth=2, n_feats=33, n_ex=75
	n_classes: 2
	Loss on training: 0.0
	Loss on test: 0.0
Trial 41
	Classifier=True, criterion=entropy
	max_depth=3,

	Loss on training: 0.0
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.0
 DESIRED: 0.052631578947368474
Trial 75
	Classifier=True, criterion=entropy
	max_depth=3, n_feats=56, n_ex=11
	n_classes: 7
	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.2857142857142857
 DESIRED: 0.0
	Loss on test: 1.0
Trial 76
	Classifier=True, criterion=gini
	max_depth=1, n_feats=22, n_ex=87
	n_classes: 8
	Loss on training: 0.6833333333333333
	Loss on test: 0.8888888888888888
Trial 77
	Classifier=True, criterion=entropy
	max_depth=1, n_feats=92, n_ex=6
	n_classes: 9
	Loss on training: 0.5
	Loss on test: 1.0
Trial 78
	Classifier=False, criterion=mse
	max_depth=3, n_feats=60, n_ex=90
	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 23742.79736723788
 DESIRED: 8997.297581431018
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 39365.75226846165
 DESIRED: 38608.026929013635
Trial 79
	Classifier

	Loss on training: 34048.86951780208
	Loss on test: 47762.84929494157
Trial 115
	Classifier=True, criterion=gini
	max_depth=3, n_feats=33, n_ex=8
	n_classes: 4
	Loss on training: 0.0
	Loss on test: 0.0
Trial 116
	Classifier=True, criterion=entropy
	max_depth=4, n_feats=22, n_ex=37
	n_classes: 7
	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.19999999999999996
 DESIRED: 0.0
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.41666666666666663
 DESIRED: 0.33333333333333337
Trial 117
	Classifier=True, criterion=gini
	max_depth=3, n_feats=22, n_ex=61
	n_classes: 5
	Loss on training: 0.16666666666666663
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.26315789473684215
 DESIRED: 0.3157894736842105
Trial 118
	Classifier=True, criterion=gini
	max_depth=4, n_feats=64, n_ex=48
	n_classes: 3
	Loss on training: 0.0
	Loss on test: 0.0
Trial 119
	Classifier=False, criterion=mse
	max_depth=4, n_feats=32, n_ex=93

	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 10763.579708824256
 DESIRED: 1029.9630251130761
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 25167.933281413687
 DESIRED: 17284.92593144478
Trial 156
	Classifier=False, criterion=mse
	max_depth=1, n_feats=33, n_ex=31
	Loss on training: 4801.97617295728
	Loss on test: 10886.422282767331
Trial 157
	Classifier=False, criterion=mse
	max_depth=2, n_feats=11, n_ex=59
	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 13416.25814834548
 DESIRED: 10186.505198421093
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 21516.446395051866
 DESIRED: 36256.11070091842
Trial 158
	Classifier=True, criterion=gini
	max_depth=4, n_feats=69, n_ex=8
	n_classes: 3
	Loss on training: 0.0
	Loss on test: 0.0
Trial 159
	Classifier=False, criterion=mse
	max_depth=4, n_feats=2, n_ex=31
	Training losses not equal:

Arrays are not almost equal to 7 decim

...

	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 5351.308952161601
 DESIRED: 188.0717220154644
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 64751.36292749515
 DESIRED: 58843.93734846978
Trial 782
	Classifier=True, criterion=entropy
	max_depth=3, n_feats=47, n_ex=6
	n_classes: 3
	Loss on training: 0.0
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.5
 DESIRED: 0.0
Trial 783
	Classifier=False, criterion=mse
	max_depth=4, n_feats=37, n_ex=39
	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 7140.2039159819
 DESIRED: 1093.2611754716756
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 53919.15476205198
 DESIRED: 71851.7658522686
Trial 784
	Classifier=False, criterion=mse
	max_depth=1, n_feats=86, n_ex=70
	Loss on training: 28725.006868488952
	Loss on test: 57895.878983277085
Trial 785
	Classifier=True, criterion=entropy
	max_depth=2, n_feats=9,

	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 5169.972595745179
 DESIRED: 1855.5630810598527
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 34692.22008706483
 DESIRED: 32699.073760706233
Trial 819
	Classifier=True, criterion=gini
	max_depth=1, n_feats=89, n_ex=70
	n_classes: 7
	Loss on training: 0.6734693877551021
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.8571428571428572
 DESIRED: 0.8095238095238095
Trial 820
	Classifier=True, criterion=entropy
	max_depth=2, n_feats=92, n_ex=27
	n_classes: 2
	Loss on training: 0.0
	Loss on test: 0.0
Trial 821
	Classifier=False, criterion=mse
	max_depth=1, n_feats=9, n_ex=73
	Loss on training: 17534.01432920888
	Loss on test: 28272.701600879875
Trial 822
	Classifier=True, criterion=gini
	max_depth=2, n_feats=4, n_ex=48
	n_classes: 8
	Loss on training: 0.5151515151515151
	Loss on test: 0.8666666666666667
Trial 823
	Classifier=True, criterion=gini
	max_dept

	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.4411764705882353
 DESIRED: 0.2647058823529411
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.6
 DESIRED: 0.4666666666666667
Trial 858
	Classifier=False, criterion=mse
	max_depth=2, n_feats=10, n_ex=34
	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 7527.337120704782
 DESIRED: 5537.166841074517
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 44732.886583863634
 DESIRED: 39066.55589774941
Trial 859
	Classifier=True, criterion=gini
	max_depth=3, n_feats=45, n_ex=45
	n_classes: 4
	Loss on training: 0.0
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.0
 DESIRED: 0.0714285714285714
Trial 860
	Classifier=False, criterion=mse
	max_depth=1, n_feats=30, n_ex=58
	Loss on training: 27907.27257249875
	Loss on test: 23671.163964737127
Trial 861
	Classifier=False, criterion=mse
	max_depth=3, n_feats=1

	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.5217391304347826
 DESIRED: 0.08695652173913049
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 0.6
 DESIRED: 0.15000000000000002
Trial 895
	Classifier=False, criterion=mse
	max_depth=4, n_feats=22, n_ex=24
	Training losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 8225.79974373306
 DESIRED: 20.35180054212683
	Test losses not equal:

Arrays are not almost equal to 7 decimals
 ACTUAL: 212408.34286740964
 DESIRED: 222210.4806730745
Trial 896
	Classifier=True, criterion=gini
	max_depth=2, n_feats=55, n_ex=66
	n_classes: 6
	Loss on training: 0.4782608695652174
	Loss on test: 0.55
Trial 897
	Classifier=True, criterion=entropy
	max_depth=1, n_feats=27, n_ex=78
	n_classes: 9
	Loss on training: 0.7407407407407407
	Loss on test: 0.8333333333333334
Trial 898
	Classifier=False, criterion=mse
	max_depth=3, n_feats=13, n_ex=94
	Training losses not equal:

Arrays are not 

KeyboardInterrupt: 

In [77]:
def my_entropy(y):
    """
    Entropy of a label sequence
    """
    hist = np.bincount(y)
    ps = hist / np.sum(hist)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

In [7]:
n_ex = np.random.randint(2, 100)
n_feats = np.random.randint(2, 100)
max_depth = np.random.randint(1, 5)
classifier = np.random.choice([True, False])
n_classes = np.random.randint(2, 10)

In [9]:
  X, y = make_blobs(n_samples=n_ex,
                              centers=n_classes,
                              n_features=n_feats,
                              random_state=1)

In [10]:
X,y

(array([[ -6.14453188, -11.18715795,   3.72292349, ...,  -3.93485061,
           7.70237032,   1.12245889],
        [ -7.02851559, -10.24460323,   2.53221005, ...,  -4.88520648,
           9.27238209,   1.38084099],
        [ -5.63642576,  -8.62115465,   3.44539663, ...,  -5.00734845,
           7.19787177,   1.89098151],
        ...,
        [-10.02637826,   0.95394909,  -3.89551406, ...,  -8.83740332,
          -8.70051817,  -3.90699393],
        [ -1.1719657 ,   6.1455629 ,  -9.9287427 , ...,   9.00247473,
          -4.01449899,  -5.48723894],
        [-10.98813991,   2.4779139 ,  -4.90068742, ...,  -8.15951149,
          -7.02521209,  -5.10894308]]),
 array([1, 1, 1, 2, 2, 2, 2, 1, 0, 2, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        2, 2, 2, 1, 1, 2, 0, 0, 1, 2, 0, 2]))

In [12]:
X.shape

(34, 49)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=1)


In [24]:
# initialize model
def loss(yp, y):
    return 1 - accuracy_score(yp, y)


criterion = np.random.choice(["entropy", "gini"])
mine = DecisionTree(classifier=True, max_depth=max_depth, criterion="entropy")

In [25]:
mine.fit(X_train,y_train)
y_pred_mine = mine.predict(X_train)
loss(y_pred_mine, y_train)

TypeError: entropy() takes 1 positional argument but 2 were given

In [ ]:
gold = DecisionTreeClassifier(
    criterion=criterion,
    max_depth=max_depth,
    splitter="best",
    random_state=1,
)

In [26]:
### 基尼系数的理解


In [1]:
import pandas as pd
play_data=pd.read_csv("datasets/play")
play_data.head()

,Day,Outlook,Temp.,Humidity,Wind,Decision
0,1,Sunny,Hot,High,Weak,No
1,2,Sunny,Hot,High,Strong,No
2,3,Overcast,Hot,High,Weak,Yes
3,4,Rain,Mild,High,Weak,Yes
4,5,Rain,Cool,Normal,Weak,Yes


In [2]:
number,_ = play_data.shape
number

14

In [4]:
import numpy as np
play_data=np.array(play_data)
play_data

array([[1, 'Sunny', 'Hot', 'High', 'Weak', 'No'],
       [2, 'Sunny', 'Hot', 'High', 'Strong', 'No'],
       [3, 'Overcast', 'Hot', 'High', 'Weak', 'Yes'],
       [4, 'Rain', 'Mild', 'High', 'Weak', 'Yes'],
       [5, 'Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
       [6, 'Rain', 'Cool', 'Normal', 'Strong', 'No'],
       [7, 'Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
       [8, 'Sunny', 'Mild', 'High', 'Weak', 'No'],
       [9, 'Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
       [10, 'Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
       [11, 'Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
       [12, 'Overcast', 'Mild', 'High', 'Strong', 'Yes'],
       [13, 'Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
       [14, 'Rain', 'Mild', 'High', 'Strong', 'No']], dtype=object)

In [5]:
X_train, y = play_data[:, 1:-1], play_data[:, -1]
X_train, y

(array([['Sunny', 'Hot', 'High', 'Weak'],
        ['Sunny', 'Hot', 'High', 'Strong'],
        ['Overcast', 'Hot', 'High', 'Weak'],
        ['Rain', 'Mild', 'High', 'Weak'],
        ['Rain', 'Cool', 'Normal', 'Weak'],
        ['Rain', 'Cool', 'Normal', 'Strong'],
        ['Overcast', 'Cool', 'Normal', 'Strong'],
        ['Sunny', 'Mild', 'High', 'Weak'],
        ['Sunny', 'Cool', 'Normal', 'Weak'],
        ['Rain', 'Mild', 'Normal', 'Weak'],
        ['Sunny', 'Mild', 'Normal', 'Strong'],
        ['Overcast', 'Mild', 'High', 'Strong'],
        ['Overcast', 'Hot', 'Normal', 'Weak'],
        ['Rain', 'Mild', 'High', 'Strong']], dtype=object),
 array(['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes',
        'Yes', 'Yes', 'Yes', 'No'], dtype=object))

### 决策树可视化
1. 使用graphviz https://graphviz.gitlab.io/_pages/Download/Download_source.html